# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.io import *
from pymodulon.plotting import *
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = path.join('..','data','processed_data')

# GO and KEGG annotations are in the 'external' folder
external_data = path.join('..','data','external')

## Load the data
You're now ready to load your IcaData object!

In [3]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   X = path.join(data_dir,'logexpset_qc_norm.csv'),
                   gene_table = path.join(data_dir,'gene_info.csv'),
                   sample_table = path.join(data_dir,'metadata.tsv'),
                   trn = path.join(external_data,'TRN.csv'),
                   imodulon_table = path.join(data_dir,'imodulon_table.csv'),
                   optimize_cutoff=True)

### Regulatory iModulons

In [4]:
ica_data.compute_trn_enrichment()

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,3,FixK2,1.017939e-21,8.143509e-21,0.188525,0.230000,0.207207,23.0,100.0,122.0,1.0
1,3,FixK,2.497987e-11,4.995974e-11,0.081967,0.303030,0.129032,10.0,33.0,122.0,1.0
2,3,FnrN,2.497987e-11,4.995974e-11,0.081967,0.303030,0.129032,10.0,33.0,122.0,1.0
3,3,AadR,2.497987e-11,4.995974e-11,0.081967,0.303030,0.129032,10.0,33.0,122.0,1.0
4,4,bll2166,3.940714e-06,7.881427e-06,0.061224,0.500000,0.109091,3.0,6.0,49.0,1.0
5,5,NifA,2.849312e-60,2.279450e-59,0.234177,0.880952,0.370000,37.0,42.0,158.0,1.0
6,5,RpoN,1.184619e-38,4.738476e-38,0.158228,0.806452,0.264550,25.0,31.0,158.0,1.0
7,6,PhyR,6.099000e-70,1.829700e-69,0.310345,1.000000,0.473684,36.0,36.0,116.0,1.0
8,11,Irr,1.691158e-33,3.382316e-33,0.313725,0.800000,0.450704,16.0,20.0,51.0,1.0
9,21,HrcA,1.607711e-08,4.823133e-08,0.039683,0.714286,0.075188,5.0,7.0,126.0,1.0


In [5]:
# First search for regulator enrichments with 2 regulators
ica_data.compute_trn_enrichment(max_regs=2,save=True)

# Next, search for regulator enrichments with just one regulator. This will supercede the 2 regulator enrichments.
ica_data.compute_trn_enrichment(max_regs=1,save=True)

regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]
print(len(ica_data.imodulon_table),'Total iModulons')
print(len(regulatory_imodulons),'Regulatory iModulons')
regulatory_imodulons

/home/gzp/anaconda3/envs/pymodulon/lib/python3.10/site-packages/pymodulon/core.py:610: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  keep_cols = self.imodulon_table.loc[


64 Total iModulons
64 Regulatory iModulons


/home/gzp/anaconda3/envs/pymodulon/lib/python3.10/site-packages/pymodulon/core.py:610: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  keep_cols = self.imodulon_table.loc[


,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs,Category,imodulon_name,Function,Sub_category,explained_variance,imodulon_type
0,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,Uncharacterized,Unchar-1,Unknown,Uncharacterized iModulons,0.001248,Uncharacterized
1,T3SS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,Structural components,T3SS,T3SS system,Secretion system,0.036405,Functional
2,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,Uncharacterized,Unchar-2,Unknown,Uncharacterized iModulons,0.001036,Uncharacterized
3,FixK2,1.017939e-21,8.143509e-21,0.188525,0.23,0.207207,23.0,100.0,122.0,1.0,Stress,FixK2-1,Oxidative adaptation,Oxidative adaptation,0.014985,Regulatory
4,bll2166,3.940714e-06,7.881427e-06,0.061224,0.50,0.109091,3.0,6.0,49.0,1.0,Amino acid,bll2166,Amino acid transport,Transportation,0.011833,Regulatory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,Uncharacterized,Unchar-20,Unknown,Uncharacterized iModulons,0.004243,Uncharacterized
60,Sulfur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,Element homeostasis,Sulfur-2,Sulfur metabolism,Sulfur homeostasis,0.004417,Functional
61,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,Uncharacterized,Unchar-21,Unknown,Uncharacterized iModulons,0.001667,Uncharacterized
62,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,Uncharacterized,Unchar-22,Unknown,Uncharacterized iModulons,0.000922,Uncharacterized


In [6]:
ica_data.rename_imodulons(regulatory_imodulons.regulator.to_dict())
ica_data.imodulon_table.head()

,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs,Category,imodulon_name,Function,Sub_category,explained_variance,imodulon_type
Unchar-1,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,Uncharacterized,Unchar-1,Unknown,Uncharacterized iModulons,0.001248,Uncharacterized
T3SS,T3SS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,Structural components,T3SS,T3SS system,Secretion system,0.036405,Functional
Unchar-2,Unchar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,Uncharacterized,Unchar-2,Unknown,Uncharacterized iModulons,0.001036,Uncharacterized
FixK2-1,FixK2,1.017939e-21,8.143509e-21,0.188525,0.23,0.207207,23.0,100.0,122.0,1.0,Stress,FixK2-1,Oxidative adaptation,Oxidative adaptation,0.014985,Regulatory
bll2166,bll2166,3.940714e-06,7.881427e-06,0.061224,0.50,0.109091,3.0,6.0,49.0,1.0,Amino acid,bll2166,Amino acid transport,Transportation,0.011833,Regulatory


### GO annotation enrichment

In [7]:
DF_GO = pd.read_csv(path.join(external_data,'GO_annotations_curated.csv'),index_col=0)
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'gene_ontology')

### KEGG annotation enrichment

In [8]:
DF_KEGG = pd.read_csv(path.join(external_data,'kegg_mapping.csv'),index_col=0)
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_module']
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')
DF_pathway_enrich = DF_pathway_enrich[~DF_pathway_enrich.kegg_id.str.contains("-")]
DF_module_enrich = DF_module_enrich[~DF_module_enrich.kegg_id.str.contains("-")]
DF_module_enrich[~DF_module_enrich.kegg_id.str.contains("-")]

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,T3SS,M00660,9.708685e-08,3.087362e-05,0.088889,0.444444,0.148148,4.0,9.0,45.0
1,T3SS,M00332,2.522482e-07,4.010746e-05,0.088889,0.363636,0.142857,4.0,11.0,45.0
2,T3SS,M00542,1.261525e-05,1.337216e-03,0.066667,0.333333,0.111111,3.0,9.0,45.0
3,FixK2-1,M00357,3.097359e-06,5.084147e-04,0.032787,0.500000,0.061538,4.0,8.0,122.0
4,FixK2-1,M00579,3.197577e-06,5.084147e-04,0.024590,1.000000,0.048000,3.0,3.0,122.0
...,...,...,...,...,...,...,...,...,...,...
66,Sulfur-1,M00239,6.091204e-04,3.228338e-02,0.102564,0.048193,0.065574,4.0,83.0,39.0
68,Bacterial_chemotaxis,M00506,2.370995e-08,3.769881e-06,0.036810,0.545455,0.068966,6.0,11.0,163.0
69,BioR,M00123,8.278723e-10,8.775447e-08,0.129032,0.800000,0.222222,4.0,5.0,31.0
70,BioR,M00573,8.278723e-10,8.775447e-08,0.129032,0.800000,0.222222,4.0,5.0,31.0


### Convert KEGG IDs to human-readable names

In [9]:
for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    text = REST.kegg_find('pathway',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except AttributeError:
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    text = REST.kegg_find('module',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except AttributeError:
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

In [10]:
#update gene names

ica_data.gene_table.loc['bll2550', 'gene_name'] = 'nadQ'
ica_data.gene_table.loc['blr0934', 'gene_name'] = 'zur'
ica_data.gene_table.loc['blr1216', 'gene_name'] = 'irr'
ica_data.gene_table.loc['blr5456', 'gene_name'] = 'psrA'
ica_data.gene_table.loc['blr2391', 'gene_name'] = 'vanR'
ica_data.gene_table.loc['bll4578', 'gene_name'] = 'nmlR'
ica_data.gene_table.loc['bll2094', 'gene_name'] = 'bioR'
ica_data.gene_table.loc['blr1118', 'gene_name'] = 'xylR'
ica_data.gene_table.loc['bll3974', 'gene_name'] = 'atu5239'
ica_data.gene_table.loc['blr5860', 'gene_name'] = 'pdxQ3'
ica_data.gene_table.loc['bll4154', 'gene_name'] = 'pdxW'
ica_data.gene_table.loc['bll6835', 'gene_name'] = 'uxuR2'
ica_data.gene_table.loc['blr1279', 'gene_name'] = 'mdcY'
ica_data.gene_table.loc['blr5870', 'gene_name'] = 'gulR'
ica_data.gene_table.loc['bll7696', 'gene_name'] = 'fnrN'
ica_data.gene_table.loc['bll7795', 'gene_name'] = 'phyR'
ica_data.gene_table.loc['blr0227', 'gene_name'] = 'phaR'
ica_data.gene_table.loc['bll2109', 'gene_name'] = 'aadR'
ica_data.gene_table.loc['bll0797', 'gene_name'] = 'mur'

## Save files

In [11]:
DF_GO_enrich['source'] = 'GO'
DF_pathway_enrich['source'] = 'KEGG pathways'
DF_module_enrich['source'] = 'KEGG modules'


DF_GO_enrich.rename({'gene_ontology':'annotation'},axis=1, inplace=True)
DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)


DF_enrichments = pd.concat([DF_GO_enrich, DF_pathway_enrich, DF_module_enrich])
DF_enrichments.to_csv(path.join(data_dir,'functional_enrichments.csv'))

# Save iModulon object

In [12]:
from pymodulon.util import explained_variance
from pymodulon.io import *

In [13]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [14]:
save_to_json(ica_data, path.join(data_dir,'bd_raw.json.gz'))